# Consolidate into a single cell to simulate single python file

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt
import json
from utils import *
import pandas as pd
import numpy as np
import pickle as pkl
import itertools
from sys import argv
from collections import defaultdict


# function to get the orientation/direction of the current edge
# edge contains (node1, node2), where node = [x,y]
def get_direction(edge):
    node1 = edge[0]
    node2 = edge[1]

    if abs(node1[0] - node2[0]) < 0.1:
        return "vertical"
    elif abs(node1[1] - node2[1]) < 0.1:
        return "horizontal"
    else:
        return "don't know"
    
# return true bend points 
# context: ogdf sometimes gives points that are not bend points in the edge list
def get_bend_points(edge_path):
    bend_points = []
    prev_direction = ""
    for i in range(len(edge_path)):
        # exit condition
        if (i == len(edge_path) - 1):
            break;
        
        # compute current edge direction
        current_path = (edge_path[i], edge_path[i+1])
        direction = get_direction(current_path)
    
        if (i == 0):
            prev_direction = direction
        if (i > 0):
            if ((prev_direction == "vertical" and direction == "horizontal") or 
                (prev_direction == "horizontal" and direction == "vertical")):
                bend_points.append(edge_path[i])
            # set the current direction to be prev for next iteration
            prev_direction = direction

    return bend_points
        

# function to produce modified graph
def create_graph(data):

    G = nx.DiGraph()

    # read in actual nodes first
    node_coordinate = {}
    # first, add the real nodes
    for node_obj in data['nodes']:
        identity = node_obj['id']
        G.add_node(identity)
        node_coordinate[identity] = (node_obj['x'], node_obj['y'], 0)

    # create virtual nodes to split an edge
    node_counter = G.number_of_nodes() -1  # node_id starts from 0
    for edge in data['edges']:
        source = edge['source']
        target = edge['target']
        # full_points_list = [list(node_coordinate[source][0:3])] + edge['path'] + [list(node_coordinate[target][0:3])]
        full_points_list = edge['path']
        # get all bend points only - ignore other points
        bend_points_list = get_bend_points(full_points_list)
        num_points = len(bend_points_list) 
        # connect from real node to first intermediate node
        inter = source
        if num_points == 0:
            G.add_edge(source, target)
            # add reverse edge
            G.add_edge(target, source)
        else:
            for i in range(num_points):
                current_node = bend_points_list[i]
                node_counter = node_counter + 1
                G.add_node(node_counter)
                node_coordinate[node_counter] = (current_node[0], current_node[1], 1)
                G.add_edge(inter, node_counter)
                # add reverse edge
                G.add_edge(node_counter, inter)
                inter = node_counter
            # last edge
            G.add_edge(inter, target)
            # add reverse edge
            G.add_edge(target, inter)
            
    # start of nodelist generation
    nodelist = []

    for node_id in G.nodes:
        # main nodes
        if node_coordinate[node_id][2] == 0:
            node_group = 0
            r = 5
            fill = 'blue'
        # bend nodes
        elif node_coordinate[node_id][2] == 1:
            node_group = 1
            r = 2
            fill = 'orange'
        # alignment nodes
        else:
            node_group = 2
            r = 2
            fill = 'red'

        cx = node_coordinate[node_id][0]
        cy = node_coordinate[node_id][1]
        node = [node_id, node_group, cx, cy, r, fill]
        nodelist.append(node)

    # start of linelist generation
    linelist = G.edges

    return G, nodelist, linelist
        

# produce the "ori" dictionary object required for the dataset
def produce_ori(ori_id, nodelist, linelist, height, width):
    ori = {}

    ori['id'] = ori_id
    ori['nodelist'] = nodelist
    ori['linelist'] = linelist
    ori['height'] = height
    ori['width'] = width

    return ori

# helper function to create initialization features
def create_feature_vector(size):
    # Generate all permutations of indices for two ones in the feature vector
    indices = list(range(size))
    permutations = itertools.permutations(indices, 2)

    # Create feature vectors with two ones at each permutation
    feature_vectors = []
    for perm in permutations:
        vector = [0] * size
        vector[perm[0]] = 1
        vector[perm[1]] = 1
        feature_vectors.append(vector)

    return feature_vectors

# Usage example
# feature_vectors = create_feature_vector(20)
def input_features(feature_vectors, nodelist):
    return feature_vectors[:len(nodelist)]

# compute BFS-ordered nodes
def sorted_list(G):

    # degree centrality
    # degCent = nx.degree_centrality(G)
    # degCent_sorted=dict(sorted(degCent.items(), key=lambda item: item[1],reverse=True))

    # find highest betweenness_centrality
    #Computing betweeness
    betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
    #Descending order sorting betweeness
    betCent_sorted=dict(sorted(betCent.items(), key=lambda item: item[1],reverse=True))
    root = list(betCent_sorted)[0]
    # root = 0
    edges = nx.bfs_edges(G, root)
    nodes = [root] + [v for u,v in edges] 
    x_idx = nodes
    x_ridx = sorted(range(len(x_idx)), key=lambda i: x_idx[i])
    return x_idx, x_ridx


# compute normalized coordinates
def norm_coord(nodelist, width, height, x_idx):
    norm_list = []
    for node in nodelist:
        norm_list.append((node[2]/(width - 50), node[3]/(height - 50)))
    norm_arr = np.array(norm_list)
    
    return norm_arr[x_idx,:];


# template for individual file generation
def file_generation(file_id, height, width, file_name):
    G, nodelist, linelist = create_graph(data)
    ori = produce_ori(file_id, nodelist, linelist, height, width)
    feature_vectors = create_feature_vector(49)
    input_feat_list = input_features(feature_vectors, nodelist)
    x_idx, x_ridx = sorted_list(G)
    norm_coord_list = norm_coord(nodelist, width, height, x_idx)

    object = {}
    object["ori"] = ori
    object["x"] = np.array(input_feat_list)
    object["x_idx"] = x_idx
    object["x_ridx"] = x_ridx
    object['pos'] = norm_coord_list
    object['len'] = G.number_of_nodes()
    object['adj'] = nx.to_numpy_array(G)
    object['graph'] = nx.to_dict_of_lists(G)

    with open(file_name, "wb") as f:
        pkl.dump(object, f)

height = 1000
width = 1000
# accept the argument
output = argv[1]
global_id = 0
for root, dirs, files in os.walk(f"main_data_folder/json/output_{output}"):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        # Do something with the file_path
        new_folder = f"main_data_folder/data/store/output_{output}"
        with open(file_path, "rb") as f:
            data = json.load(f)   
        stem_name = os.path.splitext(file_name)[0]
        new_file = os.path.join(new_folder, stem_name)
        file_generation(global_id, height, width, new_file) 
        global_id = global_id + 1


# Cell by cell processing

In [1]:
import networkx as nx 
import matplotlib.pyplot as plt
import json
from utils import *
import pandas as pd
import numpy as np
import pickle as pkl
from collections import defaultdict

In [2]:
import torch
torch.cuda.is_available()

True

# Consolidation

In [3]:
# edge contains (node1, node2), where node = [x,y]
def get_direction(edge):
    node1 = edge[0]
    node2 = edge[1]

    if abs(node1[0] - node2[0]) < 0.1:
        return "vertical"
    elif abs(node1[1] - node2[1]) < 0.1:
        return "horizontal"
    else:
        return "don't know"

In [4]:
def get_bend_points(edge_path):
    bend_points = []
    prev_direction = ""
    for i in range(len(edge_path)):
        # exit condition
        if (i == len(edge_path) - 1):
            break;
        
        # compute current edge direction
        current_path = (edge_path[i], edge_path[i+1])
        direction = get_direction(current_path)
    
        if (i == 0):
            prev_direction = direction
        if (i > 0):
            if ((prev_direction == "vertical" and direction == "horizontal") or 
                (prev_direction == "horizontal" and direction == "vertical")):
                bend_points.append(edge_path[i])
            # set the current direction to be prev for next iteration
            prev_direction = direction

    return bend_points
        


In [5]:
## function to produce modified graph

def create_graph(data):

    G = nx.DiGraph()

    # read in actual nodes first
    node_coordinate = {}
    # first, add the real nodes
    for node_obj in data['nodes']:
        identity = node_obj['id']
        G.add_node(identity)
        node_coordinate[identity] = (node_obj['x'], node_obj['y'], 0)

    # create virtual nodes to split an edge
    node_counter = G.number_of_nodes() -1  # node_id starts from 0
    for edge in data['edges']:
        source = edge['source']
        target = edge['target']
        # full_points_list = [list(node_coordinate[source][0:3])] + edge['path'] + [list(node_coordinate[target][0:3])]
        full_points_list = edge['path']
        # get all bend points only - ignore other points
        bend_points_list = get_bend_points(full_points_list)
        num_points = len(bend_points_list) 
        # connect from real node to first intermediate node
        inter = source
        if num_points == 0:
            G.add_edge(source, target)
            # add reverse edge
            G.add_edge(target, source)
        else:
            for i in range(num_points):
                current_node = bend_points_list[i]
                node_counter = node_counter + 1
                G.add_node(node_counter)
                node_coordinate[node_counter] = (current_node[0], current_node[1], 1)
                G.add_edge(inter, node_counter)
                # add reverse edge
                G.add_edge(node_counter, inter)
                inter = node_counter
            # last edge
            G.add_edge(inter, target)
            # add reverse edge
            G.add_edge(target, inter)
            
    # start of nodelist generation
    nodelist = []

    for node_id in G.nodes:
        # main nodes
        if node_coordinate[node_id][2] == 0:
            node_group = 0
            r = 5
            fill = 'blue'
        # bend nodes
        elif node_coordinate[node_id][2] == 1:
            node_group = 1
            r = 2
            fill = 'orange'
        # alignment nodes
        else:
            node_group = 2
            r = 2
            fill = 'red'

        cx = node_coordinate[node_id][0]
        cy = node_coordinate[node_id][1]
        node = [node_id, node_group, cx, cy, r, fill]
        nodelist.append(node)

    # start of linelist generation
    linelist = G.edges

    return G, nodelist, linelist
        

In [6]:
mylist = [[30.0, 30.0],
   [40.0, 28.8888888889],
   [108.8888888889, 28.8888888889],
   [108.8888888889, 57.7777777778]]

mylist = [[30,30]] + mylist
print(mylist)


[[30, 30], [30.0, 30.0], [40.0, 28.8888888889], [108.8888888889, 28.8888888889], [108.8888888889, 57.7777777778]]


In [7]:
def produce_ori(ori_id, nodelist, linelist, height, width):
    ori = {}

    ori['id'] = ori_id
    ori['nodelist'] = nodelist
    ori['linelist'] = linelist
    ori['height'] = height
    ori['width'] = width

    return ori


In [8]:
import itertools

def create_feature_vector(size):
    # Generate all permutations of indices for two ones in the feature vector
    indices = list(range(size))
    permutations = itertools.permutations(indices, 2)

    # Create feature vectors with two ones at each permutation
    feature_vectors = []
    for perm in permutations:
        vector = [0] * size
        vector[perm[0]] = 1
        vector[perm[1]] = 1
        feature_vectors.append(vector)

    return feature_vectors

# Usage example
# feature_vectors = create_feature_vector(20)

In [9]:
def input_features(feature_vectors, nodelist):
    return feature_vectors[:len(nodelist)]

In [10]:
def sorted_list(G):

    # find highest betweenness_centrality

    # degree centrality
    # degCent = nx.degree_centrality(G)
    # degCent_sorted=dict(sorted(degCent.items(), key=lambda item: item[1],reverse=True))
    #Computing betweeness
    betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
    #Descending order sorting betweeness
    betCent_sorted=dict(sorted(betCent.items(), key=lambda item: item[1],reverse=True))
    root = list(betCent_sorted)[0]
    # root = 0
    edges = nx.bfs_edges(G, root)
    nodes = [root] + [v for u,v in edges] 
    x_idx = nodes

    x_ridx = sorted(range(len(x_idx)), key=lambda i: x_idx[i])

    return x_idx, x_ridx

In [11]:
def norm_coord(nodelist, width, height, x_idx):
    norm_list = []
    for node in nodelist:
        norm_list.append((node[2]/(width - 50), node[3]/(height - 50)))
    norm_arr = np.array(norm_list)
    
    return norm_arr[x_idx,:];



## process file into pickle

In [16]:
# template for individual file generation
def file_generation(file_id, height, width, file_name):
    G, nodelist, linelist = create_graph(data)
    ori = produce_ori(file_id, nodelist, linelist, height, width)
    feature_vectors = create_feature_vector(49)
    input_feat_list = input_features(feature_vectors, nodelist)
    x_idx, x_ridx = sorted_list(G)
    norm_coord_list = norm_coord(nodelist, width, height, x_idx)

    object = {}
    object["ori"] = ori
    object["x"] = np.array(input_feat_list)
    object["x_idx"] = x_idx
    object["x_ridx"] = x_ridx
    object['pos'] = norm_coord_list
    object['len'] = G.number_of_nodes()
    object['adj'] = nx.to_numpy_array(G)
    object['graph'] = nx.to_dict_of_lists(G)

    with open(file_name, "wb") as f:
        pkl.dump(object, f)


In [43]:
height = 1000
width = 1000
output = "n75e225"
global_id = 0
for root, dirs, files in os.walk(f"main_data_folder/orthogonal/output_{output}"):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        # Do something with the file_path
        new_folder = f"main_data_folder/data/store/output_{output}"
        with open(file_path, "rb") as f:
            data = json.load(f)   
        stem_name = os.path.splitext(file_name)[0]
        new_file = os.path.join(new_folder, stem_name)
        file_generation(global_id, height, width, new_file) 
        global_id = global_id + 1


In [27]:
!pwd

/home/richard/pkgs/deepdrawing


In [38]:
num = 0
output = "n50e150"
g = {}
with open(f"main_data_folder/data/store/output_{output}/output_graph{output}s150i{num}planar","rb") as f:
    g = pkl.load(f)


In [35]:
g

{'ori': {'id': 0,
  'nodelist': [[0, 0, 1701.3227272727, 1202.2621212121, 5, 'blue'],
   [1, 0, 860.3651515152, 340.853030303, 5, 'blue'],
   [2, 0, 380.4005050505, 701.6924242424, 5, 'blue'],
   [3, 0, 1421.2227272727, 1182.2560606061, 5, 'blue'],
   [4, 0, 60.15, 1202.2277777778, 5, 'blue'],
   [5, 0, 1260.9550505051, 1382.5974747475, 5, 'blue'],
   [6, 0, 1200.9419191919, 801.6237373737, 5, 'blue'],
   [7, 0, 880.5176767677, 961.6601010101, 5, 'blue'],
   [8, 0, 1060.6247474747, 701.4934343434, 5, 'blue'],
   [9, 0, 640.4075757576, 661.5863636364, 5, 'blue'],
   [10, 0, 1280.9621212121, 1202.2974747475, 5, 'blue'],
   [11, 0, 160.4863636364, 1162.1186868687, 5, 'blue'],
   [12, 0, 1281.2055555556, 942.1560606061, 5, 'blue'],
   [13, 0, 440.5075757576, 921.8045454545, 5, 'blue'],
   [14, 0, 120.1873737374, 621.4883838384, 5, 'blue'],
   [15, 0, 1020.5247474747, 220.45, 5, 'blue'],
   [16, 0, 1060.6247474747, 961.8267676768, 5, 'blue'],
   [17, 0, 860.4217171717, 561.4863636364, 5, 'b

In [78]:
for i in range(100):
    g = {}
    with open(f"data/ortho_train_dataset_folder_preprocess/output_graphn3e4s150i{num}","rb") as f:
        g = pkl.load(f)
    max_value1 = max(g['ori']['linelist'], key=lambda x:x[0])[0]
    max_value2 = max(g['ori']['linelist'], key=lambda x:x[1])[1]
    max_value = max(max_value1, max_value2)
    if (max_value != g['len'] - 1):
        print(i)
    

{'id': 1,
 'nodelist': [[0, 0, 110.0, 66.6666666667, 5, 'blue'],
  [1, 0, 30.0, 68.8888888889, 5, 'blue'],
  [2, 0, 30.0, 30.0, 5, 'blue'],
  [3, 1, 108.8888888889, 28.8888888889, 2, 'orange']],
 'linelist': OutEdgeView([(0, 1), (1, 2), (1, 0), (2, 3), (3, 0)]),
 'height': 1000,
 'width': 1000}

In [131]:
from data import inv_transform_nodelist
ori = g['ori']
y_input = g["pos"]
y_selected = y_input[g["x_ridx"],:]
bounding_box = {}
bounding_box['left'] = 0
bounding_box['right'] = 500
bounding_box['top'] = 0
bounding_box['bottom'] = 500


pos_ori = inv_transform_nodelist(y_selected,bounding_box,2)
pos_ori = np.array(pos_ori)

visualize(pos_ori, ori=ori, name="test", test_folder=".")

# process 

In [26]:
data

{'nodes': [{'id': 0, 'x': 100.9090909091, 'y': 50.0, 'w': 20.0, 'h': 20.0},
  {'id': 1, 'x': 134.5454545455, 'y': 250.0, 'w': 20.0, 'h': 20.0},
  {'id': 2, 'x': 250.0, 'y': 300.4545454545, 'w': 20.0, 'h': 20.0},
  {'id': 3, 'x': 272.2727272727, 'y': 243.6363636364, 'w': 20.0, 'h': 20.0},
  {'id': 4, 'x': 150.0, 'y': 310.0, 'w': 20.0, 'h': 20.0},
  {'id': 5, 'x': 50.0, 'y': 360.0, 'w': 20.0, 'h': 20.0},
  {'id': 6, 'x': 90.0, 'y': 302.2727272727, 'w': 20.0, 'h': 20.0},
  {'id': 7, 'x': 374.5454545455, 'y': 53.1818181818, 'w': 20.0, 'h': 20.0},
  {'id': 8, 'x': 90.0, 'y': 360.0, 'w': 20.0, 'h': 20.0},
  {'id': 9, 'x': 50.0, 'y': 400.0, 'w': 20.0, 'h': 20.0}],
 'edges': [{'source': 0,
   'target': 5,
   'arrow': 'last',
   'path': [[90.9090909091, 50.0],
    [20.0, 50.0],
    [20.0, 360.0],
    [40.0, 360.0]]},
  {'source': 1,
   'target': 4,
   'arrow': 'last',
   'path': [[142.2727272727, 260.0], [142.2727272727, 300.0]]},
  {'source': 2,
   'target': 0,
   'arrow': 'last',
   'path': [

In [27]:
G = nx.DiGraph()

# read in actual nodes first
node_coordinate = {}
# first, add the real nodes
for node_obj in data['nodes']:
  id = node_obj['id']
  G.add_node(id)
  node_coordinate[id] = (node_obj['x'], node_obj['y'], 0)

In [28]:
# create virtual nodes to split an edge
node_counter = G.number_of_nodes()
for edge in data['edges']:
    source = edge['source']
    target = edge['target']
    # coordinates includes start point and end point
    num_points = len(edge['path'])
    # connect from real node to first intermediate node
    inter = source
    for i in range(num_points):
        # special consideration for the last edge
        node_counter = node_counter + 1
        G.add_node(node_counter)
        if (i == 0 or i == num_points-1):
            # these nodes are nodes connecting edges nodes to actual nodes
            node_coordinate[node_counter] = (edge['path'][i][0], edge['path'][i][1], 2)
        else:
            # these are edge nodes
            node_coordinate[node_counter] = (edge['path'][i][0], edge['path'][i][1], 1)
        G.add_edge(inter, node_counter)
        inter = node_counter
    # connect from last intermediate node to target
    G.add_edge(inter, target)



In [29]:
nodelist = []

for node_id in G.nodes:
    # main nodes
    if node_coordinate[node_id][2] == 0:
        node_group = 0
        r = 5
        fill = 'blue'
    # bend nodes
    elif node_coordinate[node_id][2] == 1:
        node_group = 1
        r = 5
        fill = 'orange'
    # alignment nodes
    else:
        node_group = 2
        r = 2
        fill = 'red'

    cx = node_coordinate[node_id][0]
    cy = node_coordinate[node_id][1]
    node = [node_id, node_group, cx, cy, r, fill]
    nodelist.append(node)

In [30]:
linelist = G.edges
print(linelist)

[(0, 11), (0, 22), (0, 28), (0, 34), (0, 65), (0, 69), (0, 78), (0, 100), (1, 15), (1, 38), (1, 50), (2, 17), (2, 42), (2, 57), (2, 63), (3, 32), (3, 73), (3, 82), (4, 45), (4, 76), (4, 80), (5, 54), (5, 98), (6, 60), (7, 84), (7, 87), (7, 96), (7, 104), (8, 92), (9, 94), (11, 12), (12, 13), (13, 14), (14, 5), (15, 16), (16, 4), (17, 18), (18, 19), (19, 20), (20, 21), (21, 0), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 3), (28, 29), (29, 30), (30, 31), (31, 1), (32, 33), (33, 1), (34, 35), (35, 36), (36, 37), (37, 2), (38, 39), (39, 40), (40, 41), (41, 0), (42, 43), (43, 44), (44, 1), (45, 46), (46, 47), (47, 48), (48, 49), (49, 2), (50, 51), (51, 52), (52, 53), (53, 0), (54, 55), (55, 56), (56, 6), (57, 58), (58, 59), (59, 9), (60, 61), (61, 62), (62, 1), (63, 64), (64, 4), (65, 66), (66, 67), (67, 68), (68, 7), (69, 70), (70, 71), (71, 72), (72, 1), (73, 74), (74, 75), (75, 2), (76, 77), (77, 2), (78, 79), (79, 6), (80, 81), (81, 6), (82, 83), (83, 1), (84, 85), (85, 86),

## functions for producing graph, nodelist and linelist

In [31]:
## function to produce modified graph

def create_graph(data):

  G = nx.DiGraph()

  # read in actual nodes first
  node_coordinate = {}
  # first, add the real nodes
  for node_obj in data['nodes']:
    id = node_obj['id']
    G.add_node(id)
    node_coordinate[id] = (node_obj['x'], node_obj['y'], 0)

  # create virtual nodes to split an edge
  node_counter = G.number_of_nodes()
  for edge in data['edges']:
      source = edge['source']
      target = edge['target']
      # coordinates includes start point and end point
      num_points = len(edge['path'])
      # connect from real node to first intermediate node
      inter = source
      for i in range(num_points):
          # special consideration for the last edge
          node_counter = node_counter + 1
          G.add_node(node_counter)
          if (i == 0 or i == num_points-1):
              # these nodes are nodes connecting edges nodes to actual nodes
              node_coordinate[node_counter] = (edge['path'][i][0], edge['path'][i][1], 2)
          else:
              # these are edge nodes
              node_coordinate[node_counter] = (edge['path'][i][0], edge['path'][i][1], 1)
          G.add_edge(inter, node_counter)
          inter = node_counter
      # connect from last intermediate node to target
      G.add_edge(inter, target)
      return G

# nodelist and edgelist
def produce_nodelist(G):
    nodelist = []

    for node_id in G.nodes:
        # main nodes
        if node_coordinate[node_id][2] == 0:
            node_group = 0
            r = 5
            fill = 'blue'
        # bend nodes
        elif node_coordinate[node_id][2] == 1:
            node_group = 1
            r = 5
            fill = 'orange'
        # alignment nodes
        else:
            node_group = 2
            r = 2
            fill = 'red'

        cx = node_coordinate[node_id][0]
        cy = node_coordinate[node_id][1]
        node = [node_id, node_group, cx, cy, r, fill]
        nodelist.append(node)
    return nodelist

def produce_linelist(G):
    return G.edges
        

# ori

## example of ori processing

In [32]:
# global variables
height = 600
width = 700

In [33]:
# read in data file
df = pd.read_csv("orthogonal/bounding_box.txt", header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'orthogonal/bounding_box.txt'

In [ ]:
bounding_box = {}
for index, row in df.iterrows():
  bounding_box[row[0]] = (row[1], row[2])

In [ ]:
# select bounding box
for key in bounding_box:
  if "graphn10e30s150i0" in key:
    pair = bounding_box[key]


In [ ]:

ori = {}

ori['id'] = 0
ori['nodelist'] = nodelist
ori['linelist'] = linelist
ori['height'] = height
ori['width'] = width


In [ ]:
# visualize(None, ori=object1['ori'], name="test", test_folder=".")
visualize(None, ori=ori, name="test", test_folder=".")

## function for ori

In [ ]:
def produce_ori(id, nodelist, linelist, height, width):
  ori = {}

  ori['id'] = id
  ori['nodelist'] = nodelist
  ori['linelist'] = linelist
  ori['height'] = height
  ori['width'] = width

  return ori


# input feature and sorting

In [ ]:
import itertools

def create_feature_vector(size):
    # Generate all permutations of indices for two ones in the feature vector
    indices = list(range(size))
    permutations = itertools.permutations(indices, 2)

    # Create feature vectors with two ones at each permutation
    feature_vectors = []
    for perm in permutations:
        vector = [0] * size
        vector[perm[0]] = 1
        vector[perm[1]] = 1
        feature_vectors.append(vector)

    return feature_vectors

# Usage example
feature_vectors = create_feature_vector(20)

In [ ]:
input_feature = feature_vectors[:len(nodelist)]

## function: input features

In [ ]:
def input_features(feature_vectors, nodelist):
    return feature_vectors[:len(nodelist)]

## produce bfs sorting of graph

In [ ]:
# identify most central node
data = json.load(open('orthogonal/output_n5e10/output_graphn5e10s150i0planar.json'))
G, nodelist, linelist = create_graph(data)
degCent = nx.degree_centrality(G)
degCent_sorted=dict(sorted(degCent.items(), key=lambda item: item[1],reverse=True))

#Computing betweeness
betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)

#Descending order sorting betweeness
betCent_sorted=dict(sorted(betCent.items(), key=lambda item: item[1],reverse=True))

list(betCent_sorted)[0]

In [ ]:
# produce bfs ordering of nodes
root = 0
edges = nx.bfs_edges(G, root)
nodes = [root] + [v for u,v in edges] 
x_idx = nodes

In [ ]:
# introduce order
# assume [0,1,2,3] becomes [3,1,0,2]
# then order is [2,1,3,0]
# think of where each element i would go after being sorted
# e.g. element 0 goes to position 2 (3rd place)
order = sorted(range(len(x_idx)), key=lambda i: x_idx[i])
r_idx = order

## function for x_idx and r_idx

In [ ]:
def sorted_list(G):
  root = 0
  edges = nx.bfs_edges(G, root)
  nodes = [root] + [v for u,v in edges] 
  x_idx = nodes

  r_idx = sorted(range(len(x_idx)), key=lambda i: x_idx[i])

  return x_idx, r_idx

# auxiliary

In [ ]:
nodelist

In [ ]:
# normalized coordinates
norm_coord = (pair[0]/width, pair[1]/height)

def norm_coord(nodelist):
    norm_list = []
    for node in nodelist:
        norm_list.append(node[2]/width, pair[3]/height)

    return norm_list
